In [38]:
import pandas as pd
import yfinance as yf
from selenium import webdriver
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import time
import numpy as np

In [39]:
options = Options() 
chrome_path = '/Users/carsonlloyd/Downloads/chromedriver'

In [40]:
df = pd.read_csv('stocks.csv')
df['TICKER'] = np.nan
df.head()

,NAME OF ISSUER,TITLE OF CLASS,CUSIP,(x$1000),PRN AMT,PRN,CALL,DISCRETION,MANAGER,SOLE,SHARED,NONE,TICKER
0,1347 PPTY INS HLDGS INC,COM,68244P107,443,100562,SH,NaN,SOLE,0,91267,0,9295,NaN
1,1ST CONSTITUTION BANCORP,COM,31986N102,2359,125800,SH,NaN,SOLE,0,119245,0,6555,NaN
2,1ST SOURCE CORP,COM,336901103,15320,335014,SH,NaN,SOLE,0,319352,0,15662,NaN
3,21VIANET GROUP INC,SPONSORED ADS A,90138A103,8799,1154767,SH,NaN,SOLE,0,1119600,0,35167,NaN
4,3-D SYS CORP DEL,COM NEW,88554D205,2155,264450,SH,NaN,SOLE,0,58042,0,206408,NaN


In [41]:
print(df.sample(100, random_state=42).index.to_list())

[291, 997, 2092, 432, 479, 1621, 501, 736, 2776, 2097, 2071, 2883, 3200, 2287, 1006, 3392, 1132, 2315, 1235, 170, 1206, 229, 2023, 794, 1610, 729, 298, 2478, 144, 2126, 32, 1399, 2946, 51, 1532, 2771, 2962, 2080, 1948, 2641, 393, 183, 3230, 1801, 1196, 1003, 70, 257, 239, 1146, 1538, 538, 3241, 3064, 1178, 2335, 2611, 2257, 1052, 2108, 678, 299, 2380, 554, 2932, 759, 2727, 184, 3163, 52, 2906, 2406, 1907, 1232, 857, 2854, 578, 295, 544, 781, 2590, 3159, 2475, 17, 1068, 2469, 1644, 315, 612, 1437, 1114, 1105, 1407, 1991, 2458, 790, 761, 2189, 1922, 2118]


In [42]:
cusip_list = df.sample(100, random_state=42).index.to_list()

In [43]:
# pd.set_option("display.max_rows", 200)
cusips = list(df.iloc[cusip_list]['CUSIP'])

In [44]:
driver = webdriver.Chrome(chrome_path, 
                              options=options)

driver.set_window_size(1400,1000)

In [45]:
driver.get('https://quotes.fidelity.com/mmnet/SymLookup.phtml?QUOTE_TYPE=D')

In [46]:
tickers = []
for cusip in cusips:
    try:
        driver.find_elements_by_css_selector('option')[6].click()
        time.sleep(1)
        search_bar = driver.find_element_by_name('criteria')
        search_bar.send_keys(cusip)
        search_bar.send_keys(Keys.ENTER)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source)
        tables = soup.findAll('table')
        table = tables[8].prettify()
        df = pd.read_html(table)[0]
        tickers.append(df.loc[2][1])
        driver.find_element_by_name('Reset').click()
        time.sleep(1)
    except:
        tickers.append(np.nan)
        driver.find_element_by_name('Reset').click()
        time.sleep(1)
        continue

In [47]:
a = 0
df = pd.read_csv('stocks.csv')
df['TICKER'] = np.nan
for indx in cusip_list:
    df['TICKER'].iloc[indx] = tickers[a]
    a += 1

In [37]:
driver.quit()

In [13]:
df.to_csv('stocks.csv', index = False)

In [35]:
sdf = df.loc[~df.TICKER.isna()]

In [161]:
for i, ticker in enumerate(sdf['TICKER']): 
    try:
        asset = yf.Ticker(ticker)
        shortName = asset.info['shortName']
        hist = asset.history(period="3mo")
        print(ticker, "shortName=", shortName, 
              "sdf.NAME_OF_ISSUER=", sdf['NAME OF ISSUER'].iloc[i])
    except:
        print("***Short name not found for ticker:", ticker, "not found!")

ABM shortName= ABM Industries Incorporated sdf.NAME_OF_ISSUER= ABM INDS INC
ACMR shortName= ACM Research, Inc. sdf.NAME_OF_ISSUER= ACM RESEARCH INC
AAP shortName= Advance Auto Parts Inc Advance  sdf.NAME_OF_ISSUER= ADVANCE AUTO PARTS INC
ADSW shortName= Advanced Disposal Services, Inc sdf.NAME_OF_ISSUER= ADVANCED DISP SVCS INC DEL
DOX shortName= Amdocs Limited sdf.NAME_OF_ISSUER= AMDOCS LTD
AVD shortName= American Vanguard Corporation sdf.NAME_OF_ISSUER= AMERICAN VANGUARD CORP
AP shortName= Ampco-Pittsburgh Corporation sdf.NAME_OF_ISSUER= AMPCO-PITTSBURGH CORP
***Short name not found for ticker: AMPH not found!
ARNC shortName= Arconic Inc. sdf.NAME_OF_ISSUER= ARCONIC INC
AGX shortName= Argan, Inc. sdf.NAME_OF_ISSUER= ARGAN INC
ATRC shortName= AtriCure, Inc. sdf.NAME_OF_ISSUER= ATRICURE INC
AUPH shortName= Aurinia Pharmaceuticals Inc sdf.NAME_OF_ISSUER= AURINIA PHARMACEUTICALS INC
ACB shortName= Aurora Cannabis Inc. sdf.NAME_OF_ISSUER= AURORA CANNABIS INC
CDMO shortName= Avid Bioservice

In [163]:
snm_list = ['AMPH', 'BRKL', 'CMO', 'CCL', 'CTL',
            'CDXC', 'FNHC', 'FGBI', 'HA', 'IDN',
            'LOOP', 'MBOT', 'NBR', 'NAII', 'PBR/A',
            'PPSI', 'SNBR','STE']

for ticker in snm_list:
    try:
        idx = sdf[sdf['TICKER'] == ticker].index
        asset = yf.Ticker(ticker)
        hist = asset.history(period="3mo")
        print("\tticker=", ticker, "\tsdf.NAME_OF_ISSUER=", sdf['NAME OF ISSUER'].loc[idx])
    except:
        print("***Short name not found for ticker:", ticker, "not found!")

	ticker= AMPH 	sdf.NAME_OF_ISSUER= 184    AMPHASTAR PHARMACEUTICALS IN
Name: NAME OF ISSUER, dtype: object
	ticker= BRKL 	sdf.NAME_OF_ISSUER= 479    BROOKLINE BANCORP INC DEL
Name: NAME OF ISSUER, dtype: object
	ticker= CMO 	sdf.NAME_OF_ISSUER= 538    CAPSTEAD MTG CORP
Name: NAME OF ISSUER, dtype: object
	ticker= CCL 	sdf.NAME_OF_ISSUER= 554    CARNIVAL CORP
Name: NAME OF ISSUER, dtype: object
	ticker= CTL 	sdf.NAME_OF_ISSUER= 612    CENTURYLINK INC
Name: NAME OF ISSUER, dtype: object
	ticker= CDXC 	sdf.NAME_OF_ISSUER= 678    CHROMADEX CORP
Name: NAME OF ISSUER, dtype: object
	ticker= FNHC 	sdf.NAME_OF_ISSUER= 1146    FEDNAT HLDG CO
Name: NAME OF ISSUER, dtype: object
	ticker= FGBI 	sdf.NAME_OF_ISSUER= 1178    FIRST GTY BANCSHARES INC
Name: NAME OF ISSUER, dtype: object
	ticker= HA 	sdf.NAME_OF_ISSUER= 1437    HAWAIIAN HOLDINGS INC
Name: NAME OF ISSUER, dtype: object
	ticker= IDN 	sdf.NAME_OF_ISSUER= 1610    INTELLICHECK INC
Name: NAME OF ISSUER, dtype: object
	ticker= LOOP 	sdf.NAME_O